In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import torch.utils.data as data
import matplotlib.pyplot as plt

import pickle as pkl
import glob
import os
import copy
import time
from PIL import Image

from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [3]:
def data_transformer_torch_train(): #rgb
    data_transforms = transforms.Compose([
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        ])
    return data_transforms

def data_transformer_torch_test(): #rgb
    data_transforms = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        ])
    return data_transforms

def convert_im_torch(img): # rgb
    mean= np.array( [0.485, 0.456, 0.406] ).reshape(-1,1,1)
    std = np.array( [0.229, 0.224, 0.225] ).reshape(-1,1,1)

    img = img.detach().cpu().numpy()
    img = img*std + mean
    img *= 255
    img = img.transpose(1,2,0)
    img[img>255] = 255
    img[img<0] = 0
    return img.astype(np.uint8)

class Img_Dataset(data.Dataset):
    def __init__(self, file_list, transform,labels,class_labels):
        self.file_list = file_list
        self.transform = transform
        self.labels    = labels
        self.class_labels = class_labels
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path).convert("RGB")
        img_transformed = self.transform(img)
        label = torch.tensor( self.labels[index] ,dtype=torch.float)
        class_label = self.class_labels[index]
        
        return [img_transformed,label,class_label]

In [4]:
all_class = os.listdir("../Images/periodic/all")
print(len(all_class))

55


In [5]:
remove_classes = ["stone-wall4.o"]
test_classes   = ["B5CC4D9964F342BD.o"]
all_class = [ i for i in all_class if i not in remove_classes ]

train_target_classes = [ i for i in all_class if i not in test_classes ]
test_target_classes =  [ i for i in all_class if i in test_classes ]

print(len(train_target_classes))
print(len(test_target_classes))

53
1


In [6]:
def get_paths(target_classes):
    nonperiodic_paths = []
    periodic_paths    = []
    labels = []
    class_labels = []

    for cls in target_classes:
        filepaths = glob.glob("../Images/nonperiodic/all/{}/*".format(cls))
        nonperiodic_paths+=filepaths
        class_labels += [cls]*len(filepaths)

    for cls in target_classes:
        filepaths = glob.glob("../Images/periodic/all/{}/*".format(cls))
        periodic_paths+=filepaths
        class_labels += [cls]*len(filepaths)

    labels += [0]*len(nonperiodic_paths)
    labels += [1]*len(periodic_paths)
    image_paths = nonperiodic_paths + periodic_paths
    return image_paths,labels,class_labels,target_classes

In [7]:
train_packs = get_paths(train_target_classes)
test_packs  = get_paths(test_target_classes)
train_transforms = data_transformer_torch_train()
test_transforms  = data_transformer_torch_test()

In [8]:
datasets_train = Img_Dataset(file_list=train_packs[0],transform=train_transforms,labels=train_packs[1],class_labels=train_packs[2])
datasets_test  = Img_Dataset(file_list=test_packs[0] ,transform=test_transforms,labels=test_packs[1],class_labels=test_packs[2])

dataloader_train = torch.utils.data.DataLoader(datasets_train, batch_size=8, shuffle=True,num_workers=8)
dataloader_test  = torch.utils.data.DataLoader(datasets_test , batch_size=8, shuffle=False,num_workers=8)

dataloaders  = {"train":dataloader_train,"val":dataloader_test }
dataset_sizes ={"train":len(datasets_train),"val":len(datasets_test)}

In [9]:
model = models.vgg16(pretrained=True)
model.classifier = nn.Sequential(
        nn.Linear(25088,100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(100,1) ,nn.Sigmoid())
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
"""
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential( nn.Linear(num_ftrs, 2) ,nn.Sigmoid() )
model = model.to(device

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-10-2a4415e6b680>, line 5)

In [ ]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0
num_epochs = 500
since = time.time()

acc_dict  = {"train":[],"val":[]}
loss_dict = {"train":[],"val":[]}

for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch+1,num_epochs))
    print("-"*10)
    
    for phase in ["train","val"]:
        print("---{}---".format(phase))
        sum_img = 0

        if phase == "train":
            #scheduler.step()
            model.train()
        else:
            model.eval()
            
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels,_ in dataloaders[phase]:
            sum_img += inputs.size(0)
            print("{:6}/{:6}".format(sum_img,dataset_sizes[phase]),end="\r")
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=="train"):
                preds = model(inputs)
                #_,preds = torch.max(outputs,1)
                loss = criterion(preds,labels.view_as(preds))
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)#lossは平均だから
            
        epoch_loss = running_loss / dataset_sizes[phase]#そして割る        
        loss_dict[phase].append(epoch_loss)

        print('{} Loss: {:.4f}'.format(phase, epoch_loss))

In [ ]:
with open("models/log.pkl","wb") as f:
    pkl.dump([loss_dict,acc_dict],f)

torch.save(best_model_wts,"models/best_model_wts.pt")
torch.save(model.state_dict(),"models/final_model_wts.pt")

In [18]:
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0
num_epochs = 500
since = time.time()

acc_dict  = {"train":[],"val":[]}
loss_dict = {"train":[],"val":[]}

for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch+1,num_epochs))
    print("-"*10)
    
    for phase in ["train","val"]:
        print("---{}---".format(phase))
        sum_img = 0

        if phase == "train":
            #scheduler.step()
            model.train()
        else:
            model.eval()
            
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels,_ in dataloaders[phase]:
            sum_img += inputs.size(0)
            print("{:6}/{:6}".format(sum_img,dataset_sizes[phase]),end="\r")
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=="train"):
                preds = model(inputs)
                #_,preds = torch.max(outputs,1)
                loss = criterion(preds,labels.view_as(preds))
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)#lossは平均だから
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / dataset_sizes[phase]#そして割る
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        loss_dict[phase].append(epoch_loss)
        acc_dict[phase].append(epoch_acc.cpu().numpy())

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        
        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        if epoch % 50 == 0:
            torch.save(model.state_dict(),"models/model_{}.pt".format(epoch))
            
with open("models/log.pkl","wb") as f:
    pkl.dump([loss_dict,acc_dict],f)

torch.save(best_model_wts,"models/best_model_wts.pt")
torch.save(model.state_dict(),"models/final_model_wts.pt")

Epoch 1/500
----------
---train---
train Loss: 0.7261 Acc: 0.0000
---val---
val Loss: 0.7247 Acc: 0.0000
Epoch 2/500
----------
---train---
train Loss: 0.7266 Acc: 0.0000
---val---
val Loss: 0.7247 Acc: 0.0000
Epoch 3/500
----------
---train---


KeyboardInterrupt: 

In [17]:
preds.shape

torch.Size([6, 1])

In [ ]:
outpts